In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge

In [3]:
comp_df = pd.read_csv('train.csv')
original_df = pd.read_csv('synthetic_road_accidents_100k.csv')
test_df = pd.read_csv('test.csv')

In [4]:
comp_df.tail(10)

,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
517744,517744,highway,4,0.38,60,night,clear,True,False,afternoon,True,False,1,0.42
517745,517745,highway,4,0.02,35,daylight,clear,True,True,morning,False,False,0,0.02
517746,517746,rural,4,0.53,70,dim,rainy,True,True,evening,False,False,3,0.53
517747,517747,urban,3,0.39,35,dim,foggy,True,True,morning,False,False,0,0.15
517748,517748,rural,2,0.53,70,dim,foggy,True,True,afternoon,True,True,2,0.37
517749,517749,highway,4,0.10,70,daylight,foggy,True,True,afternoon,False,False,2,0.32
517750,517750,rural,4,0.47,35,daylight,rainy,True,True,morning,False,False,1,0.26
517751,517751,urban,4,0.62,25,daylight,foggy,False,False,afternoon,False,True,0,0.19
517752,517752,highway,3,0.63,25,night,clear,True,False,afternoon,True,True,3,0.51
517753,517753,highway,2,0.31,45,dim,rainy,False,True,afternoon,True,True,2,0.22


In [5]:
original_df.tail()

,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
99995,rural,2,0.61,60,dim,foggy,False,True,evening,False,False,1,0.54
99996,rural,2,0.52,35,dim,foggy,True,True,afternoon,False,False,1,0.18
99997,rural,2,0.08,70,daylight,clear,False,False,evening,True,False,1,0.20
99998,rural,1,0.83,70,dim,foggy,False,True,morning,False,True,0,0.58
99999,rural,2,0.01,45,daylight,foggy,False,True,evening,True,False,2,0.14


In [6]:
equal_data = comp_df.equals(original_df)
print(equal_data)


False


In [7]:
print("comp train data shape: ",comp_df.shape)
print("original data shape: ",original_df.shape)

comp train data shape:  (517754, 14)
original data shape:  (100000, 13)


In [ ]:
comp_df.drop('id', axis=1,inplace=True)
test_df.drop('id', axis=1,inplace=True)

In [9]:
comp_df.head(1)

,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,urban,2,0.06,35,daylight,rainy,False,True,afternoon,False,True,1,0.13


In [10]:
comp_df['is_original'] = 0
original_df['is_original']=1

In [11]:
full_train_df = pd.concat([comp_df,original_df],ignore_index=1)

In [14]:
full_train_df.shape

(617754, 14)

In [15]:
full_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 617754 entries, 0 to 617753
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   road_type               617754 non-null  object 
 1   num_lanes               617754 non-null  int64  
 2   curvature               617754 non-null  float64
 3   speed_limit             617754 non-null  int64  
 4   lighting                617754 non-null  object 
 5   weather                 617754 non-null  object 
 6   road_signs_present      617754 non-null  bool   
 7   public_road             617754 non-null  bool   
 8   time_of_day             617754 non-null  object 
 9   holiday                 617754 non-null  bool   
 10  school_season           617754 non-null  bool   
 11  num_reported_accidents  617754 non-null  int64  
 12  accident_risk           617754 non-null  float64
 13  is_original             617754 non-null  int64  
dtypes: bool(4), float64(

In [16]:
full_train_df.isnull().sum()

road_type                 0
num_lanes                 0
curvature                 0
speed_limit               0
lighting                  0
weather                   0
road_signs_present        0
public_road               0
time_of_day               0
holiday                   0
school_season             0
num_reported_accidents    0
accident_risk             0
is_original               0
dtype: int64

In [17]:
full_train_df.columns

Index(['road_type', 'num_lanes', 'curvature', 'speed_limit', 'lighting',
       'weather', 'road_signs_present', 'public_road', 'time_of_day',
       'holiday', 'school_season', 'num_reported_accidents', 'accident_risk',
       'is_original'],
      dtype='object')

In [18]:
full_train_df['road_type'].value_counts()

road_type
highway    207096
rural      205846
urban      204812
Name: count, dtype: int64

In [19]:
full_train_df['lighting'].value_counts()

lighting
dim         217388
daylight    211202
night       189164
Name: count, dtype: int64

In [20]:
full_train_df['weather'].value_counts()

weather
foggy    214737
clear    212817
rainy    190200
Name: count, dtype: int64

In [22]:
full_train_df['time_of_day'].value_counts()

time_of_day
morning      206784
evening      206109
afternoon    204861
Name: count, dtype: int64